In [1]:
import pandas as pd


import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

url = URL.create(
drivername="postgresql",
)

engine = create_engine(url)


## Запит по ЕМЗ 2020-2022 роки

In [6]:
sql = \
"""
SELECT 
       dle.registration_area AS Область_реєстрації,
       dle.edrpou AS ЕДРПОУ,
       rse.id AS ID_ЕМЗ,
       rse.packet_number AS Пакет,
       rse.patient_id AS ID_Пацієнта,
       rse.admission_source AS Підстава_госпіталізації,
       rse.discharge_mode AS Результат_госпіталізації,
       rse.starts  AS Дата_госпіталізації,       
       rse.ends AS Дата_виписки,
       rse.principal_diagnosis AS Діагноз_основний,
       rse.add_diagnoses AS Діагнози_додаткові,
       rse.action_references AS Проведені_інтервенції,
       rse.person_gender AS Стать,
       rse.age_years AS Вік,
       rse.age_days AS Вік_до_року
  FROM analytics.rds_smd_events_ehealth_2022 AS rse
  LEFT JOIN analytics.dwh_legal_entities_edrpou_view AS dle USING (edrpou)
 WHERE rse.is_correct
   AND rse.ends BETWEEN '2021-01-01' AND '2022-12-31'
   AND rse.packet_number IN ('4', '8')
   AND rse.admission_source = 'born_in_LE'
"""

df = pd.read_sql(sql, con=engine)
df.to_csv("./data/main.csv", index=False)


ГОТОВО !!!


## Запит по ЕМЗ 2023 роки

In [ ]:
sql_main = \
"""
WITH 
   ed AS (
SELECT *
  FROM analytics.rds_pmg_events_2023 
 INNER JOIN analytics.rds_pmg_events_checks_2023 
       USING(id)
  WHERE is_correct
    AND starts >= '2023-01-01'
    AND packet_number = '1'
   )
  
SELECT le.edrpou AS "ЕДРПОУ",
       le.kwd_name AS "Назва_Закладу",
       le.registration_area AS "Область_реєстрації",
       ed.patient_id  AS "ID_Пацієнта",
       ed.admission_source  AS "Підстава_госпіталізації",
       ed.discharge_mode AS "Результат_госпіталізації",
       ed.starts  AS "Дата_госпіталізації",
       ed.ends AS "Дата_виписки",
       ed.length_of_stay AS "Тривалість_лікування",
       ed.principal_diagnosis AS "Діагноз_основний", 
       ed.add_diagnoses AS "Діагнози_додаткові",
       ed.action_references AS "Проведені_інтервенції",
       ed.person_gender AS "Стать",
       ed.age_years AS "Вік",
       ed.age_days AS "Вік_до_року"
  FROM ed 
  LEFT JOIN analytics.dwh_legal_entities_edrpou_view AS le USING (edrpou)
"""
df = pd.read_sql(sql_main, con=engine)
df.to_csv("./data/main.csv", index=False)

## Запит по ЕМЗ 2021-2023 роках

In [3]:
sql_all = \
"""
WITH 
   Y23 AS (
SELECT *
FROM analytics.rds_pmg_events_2023
 INNER JOIN analytics.rds_pmg_events_checks_2023 
       USING(id)
 LEFT JOIN analytics.dwh_legal_entities_edrpou_view USING (edrpou)
),
    Y22 AS (
SELECT *
  FROM analytics.rds_smd_events_ehealth_2022
  LEFT JOIN analytics.dwh_legal_entities_edrpou_view USING (edrpou)
),
    Y22_Y23 AS (
SELECT registration_area AS Область_реєстрації,
      edrpou AS ЕДРПОУ,
      kwd_name AS Назва_Закладу,
      id AS ID_ЕМЗ,
      packet_number AS Пакет,
      patient_id AS Пацієнт,
      admission_source AS Підстава_госпіталізації,
      discharge_mode AS Результат_госпіталізації,
      starts  AS Дата_госпіталізації,       
      ends AS Дата_виписки,
      length_of_stay AS Тривалість_лікування,
      principal_diagnosis AS Діагноз_основний,
      add_diagnoses AS Діагнози_додаткові,
      action_references AS Проведені_інтервенції,
      person_gender AS Стать,
      age_years AS Вік,
      age_days AS Вік_до_року,
      is_correct
 FROM Y22
UNION 
SELECT registration_area AS Область_реєстрації,
      edrpou AS ЕДРПОУ,
      kwd_name AS Назва_Закладу,
      id AS ID_ЕМЗ,
      packet_number AS Пакет,
      patient_id AS Пацієнт,
      admission_source AS Підстава_госпіталізації,
      discharge_mode AS Результат_госпіталізації,
      starts  AS Дата_госпіталізації,       
      ends AS Дата_виписки,
      length_of_stay AS Тривалість_лікування,
      principal_diagnosis AS Діагноз_основний,
      add_diagnoses AS Діагнози_додаткові,
      action_references AS Проведені_інтервенції,
      person_gender AS Стать,
      age_years AS Вік,
      age_days AS Вік_до_року,
      is_correct
  FROM Y23  
  )
 
SELECT *         
  FROM Y22_Y23
 WHERE is_correct
    AND Пацієнт = 'E90E23C9CE014B66F5D2B72477231B7EC842D7D8A316211886398E7AD2131A5C'
  ORDER BY  Дата_госпіталізації
 """
 
df = pd.read_sql(sql_all, con=engine)
df.to_csv("./data/main.csv", index=False) 
 